###Prototype pricing model

Task is to build a fucntion that is able to use the previously created data in the form of several input parameters and give vack the value of contract.

We're creating a function called **price_contract()** that calculates the value of contract(profit/loss) obtained by undertaking trades(buying,storing,selling of natural gas,storage of gas, injection/withdrawal) on given dates for a contract.

This function takes in 8 inputs:
* **in_dates:** A list of dates on which the gas is being injected into the storage facility.
* **in_prices:** A list of prices of gas on each of the injection dates
* **out_dates:** A list of dates on which the gas is being withdrawn from the storage facility.
* **out_prices:** A list of prices of gas on each of the withdrawal dates
* **rate:** The rate of gas in cubic feet per day
* **storage_cost_rate:** A fixed monthly fee to store the gas
* **total_vol:** The total volume of gas in cubic feet that can be stored.
* **injection_withdrawal_cost_rate:** The injection/withdrawal cost of gas in dollars per cubic feet

In [9]:
#Implementing the function
from datetime import date
import math

def price_contract(in_dates,in_prices,out_dates,out_prices,
                   rate,storage_cost_rate,total_vol,
                   injection_withdrawal_cost_rate):

  volume = 0
  buy_cost = 0
  cash_in = 0
  injection_cost = 0
  withdrawal_cost = 0
  last_date = min(min(in_dates),min(out_dates))

  all_dates = sorted(set(in_dates + out_dates))    #ensure that dates are in sequence

  for i in range(len(all_dates)):
    start_date = all_dates[i]         #processing code for each dates

    if start_date in in_dates:  # Check if the date is in in_dates
      if volume <= total_vol - rate:   #inject on these dates and sum up cash flows
        volume += rate
        buy_cost += rate * in_prices[in_dates.index(start_date)]  #cost to purchase gas
        injection_cost += rate * injection_withdrawal_cost_rate #update injection cost
        buy_cost += injection_cost
        print(f'Injected gas on {start_date} at a price of {in_prices[in_dates.index(start_date)]}')
      else:
        print(f"Injection is not possible on date {start_date} as there is insufficient space in the storage facility") #no need to inject when rate is greater than total volume minus volume
    elif start_date in out_dates:    #withdraw on these dates and sum cash flows
      if volume >= rate:
        volume -= rate
        cash_in += rate * out_prices[out_dates.index(start_date)]
        withdrawal_cost += rate * injection_withdrawal_cost_rate  #update withdrawal cost
        cash_in -= withdrawal_cost
        print(f'Extracted gas on {start_date} at a price of {out_prices[out_dates.index(start_date)]}')
      else:
        print(f'Extraction is not possible on date {start_date} as there is insufficient volume of gas')   #can't draw more gas than actually stored

  store_cost = math.ceil((max(out_dates) - min(in_dates)).days / 30) * storage_cost_rate
  profit = cash_in - buy_cost - store_cost
  return profit

How the function actually flows?

ensures that all the dates are in sequence and in ascending order → iterates over all the dates & calculates the cash flows on each date ⟶ if current date is injection date then injects gas into the storage facility & calculates the cost to store the gas, purchase the gas, injection cost → if current date is withdrawal date then witdraws from the storage facility & calculates cash inflow from selling the gas, cost to store remaining gas, withdrawal cost → finally returns the net profit/loss by storage cost & cost to purchase gas from cash inflow from selling the gas

In [10]:
#Lets check with an example usage of the function

in_dates = [date(2022, 1, 1), date(2022, 2, 1), date(2022, 2, 21), date(2022, 4, 1)] #injection dates
in_prices = [20, 21, 20.5, 22]#prices on the injection days
out_dates = [date(2022, 1, 27), date(2022, 2, 15), date(2022, 3, 20), date(2022, 6, 1)] # extraction dates
out_prices = [23, 19, 21, 25] # prices on the extraction days
rate = 100000  # rate of gas in cubic feet per day
storage_cost_rate = 10000  # total volume in cubic feet
injection_withdrawal_cost_rate = 0.0005  # $/cf
max_storage_volume = 500000 # maximum storage capacity of the storage facility
result = price_contract(in_dates, in_prices, out_dates, out_prices, rate, storage_cost_rate, max_storage_volume, injection_withdrawal_cost_rate)
print()
print(f"The value of the contract is: ${result}")

Injected gas on 2022-01-01 at a price of 20
Extracted gas on 2022-01-27 at a price of 23
Injected gas on 2022-02-01 at a price of 21
Extracted gas on 2022-02-15 at a price of 19
Injected gas on 2022-02-21 at a price of 20.5
Extracted gas on 2022-03-20 at a price of 21
Injected gas on 2022-04-01 at a price of 22
Extracted gas on 2022-06-01 at a price of 25

The value of the contract is: $389000.0
